# Advanced pandas for Data Analysis

## Agenda

 - Intro
 - Data I/O
 - Group by syntax & analysis
 - Data Joins
 - Recap

## Intro

### Learning objective(s)

 - Perform advanced data analysis and summarization
 - Joins and different types of joins
 
### Packages

 - Pandas ([documentation](https://pandas.pydata.org/pandas-docs/stable/))
 - Numpy ([documentation](https://docs.scipy.org/doc/))
 - Matplotlib ([documentation](https://matplotlib.org/api/api_overview.html))

## Data I/O

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# A version of the titanic data set contianing null values, 
# and other data quality issues
observations = pd.read_csv('resources/titanic.csv')
print(observations.columns)
# Renaming: We can also convert the variable names to be a little more consistent and user friendly
observations.columns = list(map(lambda x: str(x).lower(), 
                                observations.columns))

# Male columns
observations['male'] = observations['sex'] == 'male'
print(observations.columns)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['passengerid', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp',
       'parch', 'ticket', 'fare', 'cabin', 'embarked', 'male'],
      dtype='object')


In [4]:
observations.head(10)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,male
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,True
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,True
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,True
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,False
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,False


## Group by syntax & analysis

Groupbys and aggregates allow us to analyze similar observations, to better understand a group of observations.

Here we are going to use the `groupby` function: [Documentation here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)

> 1. __STEP 1 (SPLIT):__ Splitting is completed by creating a "GroupBy" object which basically just means separating out the data into constituent groups. We will focus on creating GroupBy objects in the following way: 
>> a. `grouped = obj.groupby(key)`<br>
>> b. `grouped = obj.groupby([key1, key2])`
>> - Here, the `obj` is usually a Pandas `DataFrame` object and the `key` is most commonly one of the following:<br>
>> > a. Column name<br>
>> > b. List of Column names<br> 
>> - In practice, the "group keys" will refer to the labels of each group (i.e. "A" and "B" are the group keys for Group "A" and Group "B" and "Group" would be passed as the `key` argument into the `groupby` function)

In [5]:
# Here we are creating a groubpy object grouped by the pclass column as suggested above
# obj: observations DataFrame
# key: pclass columm
pclass_grouped_object = observations.groupby(by='pclass')
pclass_grouped_object

> 2. __STEP 2 (APPLY):__ Now that a GroupBy object has been created, we can perform various "apply" methods to the object, here we will focus on aggregation:
>> __Aggregation:__ Aggregation returns a new DataFrame and can be completed in this fashion: `grouped.agg` or `grouped.aggregate` where an aggregation parameter is passed. We can also quickly perform less flexible summation methods using the following aggregation functions associated with grouby objects:
>>> - sum()
>>> - mean()
>>> - median()
>>> - max()
>>> - min()<br>

In [8]:
#Example of using the agg method
pclass_grouped_object.agg('mean')

,passengerid,survived,age,sibsp,parch,fare,male
pclass,,,,,,,
1,461.597222,0.629630,38.233441,0.416667,0.356481,84.154687,0.564815
2,445.956522,0.472826,29.877630,0.402174,0.380435,20.662183,0.586957
3,439.154786,0.242363,25.140620,0.615071,0.393075,13.675550,0.706721


In [9]:
#Example of just calling the mean function
pclass_grouped_object.mean()

,passengerid,survived,age,sibsp,parch,fare,male
pclass,,,,,,,
1,461.597222,0.629630,38.233441,0.416667,0.356481,84.154687,0.564815
2,445.956522,0.472826,29.877630,0.402174,0.380435,20.662183,0.586957
3,439.154786,0.242363,25.140620,0.615071,0.393075,13.675550,0.706721


In [10]:
# Let's look at the average for numerical variables, within each `pclass`
#'as_index = False' allows your key to remain its own column in dataframe rather than an index value
pclass_grouped_observations = observations.groupby(by=['pclass'], 
                                                   as_index=False).mean() 
print('Pclass grouped observations:')
display(pclass_grouped_observations)

Pclass grouped observations:


,pclass,passengerid,survived,age,sibsp,parch,fare,male
0,1,461.597222,0.629630,38.233441,0.416667,0.356481,84.154687,0.564815
1,2,445.956522,0.472826,29.877630,0.402174,0.380435,20.662183,0.586957
2,3,439.154786,0.242363,25.140620,0.615071,0.393075,13.675550,0.706721


In [10]:
# We can also bin age, evaluate how age effects other variables
# The pandas cut method allows us to bin values into discrete intervals
# documentation: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.cut.html

max_age = observations['age'].max()
observations['age_bin'] = pd.cut(observations['age'],
                                     bins = range(0,int(max_age + 11),10), #creating bins that run in intervals of 10
                                     right = False, #specifies to not include right most value of interval in bin
                                     labels = range(0,int(max_age + 1),10)) #can provide list of labels for bins

observations[['age', 'age_bin']].head(10)



,age,age_bin
0,22.0,20
1,38.0,30
2,26.0,20
3,35.0,30
4,35.0,30
5,NaN,NaN
6,54.0,50
7,2.0,0
8,27.0,20
9,14.0,10


In [16]:
# Let's explore some of the other options available to us with the groupby objects

# We can perform multiple aggregations 
# by passing them as a list into the agg function
# We also have some flexibility in either entering a string,
# a numpy method or built-in python functions
age_bin_grouped_observations = observations.groupby(by=['age_bin'], as_index=False).agg(['mean', 'std'])
print('age_bin_grouped_observations: ')
display(age_bin_grouped_observations.head(3))

age_bin_grouped_observations: 


passengerid              survived              pclass            \
               mean         std      mean       std      mean       std   
age_bin                                                                   
0        424.741935  276.898451  0.612903  0.491062  2.629032  0.579255   
10       444.362745  277.146485  0.401961  0.492715  2.470588  0.779767   
20       433.231818  258.674627  0.350000  0.478057  2.450000  0.747752   

               age               sibsp               parch            \
              mean       std      mean       std      mean       std   
age_bin                                                                
0         4.083387  2.834747  1.854839  1.607837  1.403226  0.526656   
10       16.779412  2.263107  0.666667  1.213238  0.470588  0.779767   
20       24.534091  2.807763  0.322727  0.604724  0.250000  0.659493   

              fare                 male            
              mean        std      mean       std  
age_bin                                            
0        30.576679  25.195357  0.516129  0.503819  
10       32.535132  50.481289  0.558824  0.498980  
20       27.278937  44.196383  0.672727  0.470288

In [18]:
age_bin_grouped_observations['passengerid']

,mean,std
age_bin,,
0,424.741935,276.898451
10,444.362745,277.146485
20,433.231818,258.674627
30,472.449102,253.616532
40,465.606742,252.477750
50,440.187500,245.893391
60,433.736842,219.551270
70,496.500000,323.545824
80,631.000000,NaN


In [14]:
# If you would like to move to single index
age_bin_grouped_observations.columns = ["_".join(i) for i in age_bin_grouped_observations.columns]
age_bin_grouped_observations.head(3)

,passengerid_mean,passengerid_std,survived_mean,survived_std,pclass_mean,pclass_std,age_mean,age_std,sibsp_mean,sibsp_std,parch_mean,parch_std,fare_mean,fare_std,male_mean,male_std
age_bin,,,,,,,,,,,,,,,,
0,424.741935,276.898451,0.612903,0.491062,2.629032,0.579255,4.083387,2.834747,1.854839,1.607837,1.403226,0.526656,30.576679,25.195357,0.516129,0.503819
10,444.362745,277.146485,0.401961,0.492715,2.470588,0.779767,16.779412,2.263107,0.666667,1.213238,0.470588,0.779767,32.535132,50.481289,0.558824,0.498980
20,433.231818,258.674627,0.350000,0.478057,2.450000,0.747752,24.534091,2.807763,0.322727,0.604724,0.250000,0.659493,27.278937,44.196383,0.672727,0.470288


In [19]:
# We can also group according to different aggregation metrics per value
# Here we pass through a dictionary to specify the aggregation metric by column
bin_pclass_grouped = observations.groupby(['age_bin','pclass']).aggregate({'survived':'mean',
                                                                           'fare':'median',
                                                                           'sibsp':'max'}).reset_index()
bin_pclass_grouped = bin_pclass_grouped.rename(columns = {'survived':'survived_mean',
                                                          'fare':'fare_median',
                                                          'sibsp':'sibsp_max'})

print('View groupby by both age_bin and pclass for different aggregation metrics')
display(bin_pclass_grouped.head(3))

View groupby by both age_bin and pclass for different aggregation metrics


,age_bin,pclass,survived_mean,fare_median,sibsp_max
0,0,1,0.666667,151.55,1.0
1,0,2,1.000000,26.25,2.0
2,0,3,0.452381,21.55,5.0


### Lab 1

Please do lab excercise 1 in the adjoining lab notebook

## Data joins

It is often helpful to join multiple tables together for analysis. Generally, we'd join tables from different datasets, for illustrative purposes we'll join our dataset with itself

We will walk through 4 different types of joins
* **Inner Join:** “Inner join produces only the set of records that match in both Table A and Table B.”
* **Outer Join:** “Full outer join produces the set of all records in Table A and Table B, with matching records from both sides where available. If there is no match, the missing side will contain null.”
* **Right Join:** "Right outer join produces a complete set of records from Table B, with the matching records (where available) in Table A. If there is no match, the right side will contain null.”
* **Left Join:** "Left outer join produces a complete set of records from Table A, with the matching records (where available) in Table B. If there is no match, the right side will contain null.”

In [20]:
# First let's create an aggregate table to join with
pclass_grouped_observations = observations.groupby('pclass', as_index=False).median()[['pclass', 'fare', 'age']]
pclass_grouped_observations


,pclass,fare,age
0,1,60.2875,37.0
1,2,14.2500,29.0
2,3,8.0500,24.0


In [25]:
pclass_1_and_2 = pclass_grouped_observations[pclass_grouped_observations['pclass']<3]
pclass_1_and_2

,pclass,fare,age
0,1,60.2875,37.0
1,2,14.2500,29.0


### Inner Join: 

Inner join produces only the set of records that match in both Table A and Table B.

In [22]:
# Now we can join our tables together using merge function. We will start with inner join
merged_observations_inner = pd.merge(observations, 
                                   pclass_1_and_2,
                                   how = 'inner',
                                   on='pclass', 
                                   suffixes=('', '_pclass_median')) #when two columns have same name, this will specify how to differentiate

#Unique values using column on which we did our inner join 
# will only include values 
#that are present in both the left and right DataFrames
display(merged_observations_inner.sample(3,random_state=42))
print('\n\n unique values of pclass in new inner join df:',merged_observations_inner.pclass.unique())


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,male,age_bin,fare_pclass_median,age_pclass_median
209,863,1,1,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0,0,17466,25.9292,D17,S,False,40,60.2875,37.0
280,317,1,2,"Kantor, Mrs. Sinai (Miriam Sternin)",female,24.0,1,0,244367,26.0000,NaN,S,False,20,14.2500,29.0
33,175,0,1,"Smith, Mr. James Clinch",male,56.0,0,0,17764,30.6958,A7,C,True,50,60.2875,37.0




 unique values of pclass in new inner join df: [1 2]


### **Outer Join:** “Full outer join produces the set of all records in Table A and Table B, with matching records from both sides where available. If there is no match, the missing side will contain null.”

In [ ]:
# We are going to add one more row to pclass_1_and_2 to show full example of an outer join

pclass_1_and_2

# the dataframe object allows you to take different structures and create pandas dataframes
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
add_on = pd.DataFrame([[4,25.63,35.0]],columns=['pclass','fare','age'],index=[2])
print('Dataframe to add on')
display(add_on)

print('\n\nOriginal dataframe:')
display(pclass_1_and_2)

# concat allows you to concatenate two dataframes together
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
pclass_1_2_and_4 = pd.concat([pclass_1_and_2,add_on],axis=0)

print('\n\nFinal')
pclass_1_2_and_4




In [ ]:
# Now we can join our tables together using merge function. 
# This time we will be doing an outer join
merged_observations_outer = pd.merge(observations, 
                                   pclass_1_2_and_4,
                                   how = 'outer',
                                   on='pclass', 
                                   suffixes=('', '_pclass_median')) 

# Unique values using column on which we did our outer join will now include all values 
# that are present in both the left and right DataFrames
display(merged_observations_outer[merged_observations_outer.pclass.isin([3,4])]\
        .sort_values(by='pclass',ascending = False).head(3))
print('\n\nunique values of pclass in new outer join df:',merged_observations_outer.pclass.unique())


### **Right Join:** "Right outer join produces a complete set of records from Table B, with the matching records (where available) in Table A. If there is no match, the right side will contain null.”

In [ ]:
# This time we will be doing a right outer join
merged_observations_right = pd.merge(observations, 
                                   pclass_1_2_and_4,
                                   how = 'right',
                                   on='pclass', 
                                   suffixes=('', '_pclass_median')) 

# Unique values using column on which we did our right outer join will now include all values 
# that are present in the right DataFrame
display(merged_observations_right.sort_values(by='pclass',ascending = False).head(3))
print('\n\nunique values of pclass in new right join df:',merged_observations_right.pclass.unique())


### **Left Join:** "Left outer join produces a complete set of records from Table A, with the matching records (where available) in Table B. If there is no match, the right side will contain null.”

In [ ]:
# This time we will be doing a left outer join
merged_observations_left = pd.merge(observations, 
                                   pclass_1_2_and_4,
                                   how = 'left',
                                   on='pclass', 
                                   suffixes=('', '_pclass_median')) 

# Unique values using column on which we did our right outer join will now include all values 
# that are present in the left DataFrame
display(merged_observations_left[merged_observations_left.pclass == 3].head(3))
print('\n\nunique values of pclass in new left join df:',merged_observations_left.pclass.unique())


### Lab 2

Please do lab excercise 2 in the adjoining lab notebook

## Recap

### Learning objectives

 - Perform advanced data analysis and summarization, including groupbys, and joins
 
### Launch questions

 - What is the syntax to group by a column in pandas?
 - What is a potential use case for joins in your workflow?